In [560]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imageio

In [561]:
def plot_picture(pic):
    #fig=plt.figure(figsize=(size,size))
    plt.imshow(pic)
    plt.show()

In [562]:
w = 1000
h = 600
background = np.zeros((h,w,3), dtype='uint8')

In [563]:
def rotateAndScale(img, scaleFactor = 0.5, degreesCCW = 30):
    (oldY,oldX) = img.shape[:-1] #note: numpy uses (y,x) convention but most OpenCV functions use (x,y)
    M = cv2.getRotationMatrix2D(center=(oldX/2,oldY/2), angle=degreesCCW, scale=scaleFactor) #rotate about center of image.

    #choose a new image size.
    newX,newY = oldX*scaleFactor,oldY*scaleFactor
    #include this if you want to prevent corners being cut off
    r = np.deg2rad(degreesCCW)
    newX,newY = (abs(np.sin(r)*newY) + abs(np.cos(r)*newX),abs(np.sin(r)*newX) + abs(np.cos(r)*newY))

    #the warpAffine function call, below, basically works like this:
    # 1. apply the M transformation on each pixel of the original image
    # 2. save everything that falls within the upper-left "dsize" portion of the resulting image.

    #So I will find the translation that moves the result to the center of that region.
    (tx,ty) = ((newX-oldX)/2,(newY-oldY)/2)
    M[0,2] += tx #third column of matrix holds translation, which takes effect after rotation.
    M[1,2] += ty

    rotatedImg = cv2.warpAffine(img, M, dsize=(int(newX),int(newY)))
    return rotatedImg


In [564]:
lover_2 = cv2.imread('1.png')
lover_1 = cv2.flip(cv2.imread('2.png'),1)

heart = cv2.resize(cv2.imread('heart.jpeg'), (w,h))

In [565]:
def overlay_image(background, lover_1, lover_2, dist_c = 100):
    h, w, _ = background.shape
    overlay = background.copy()
    
    h_0 = int(h*0.9)
    y = int(h*0.1)
    
    fx = h_0 / lover_1.shape[0]
    lover_1 = cv2.resize(lover_1, (0,0), fx = fx, fy = fx)
    h1, w1, _ = lover_1.shape
    x2 = w // 2 - dist_c
    y1 = y
    h1 = min(h1, h - y1)
    w1 = min(w1, x2)
    ind = lover_1[:h1,-w1:] != 0
    overlay[y1:y1 + h1, x2 - w1:x2][ind] = lover_1[:h1,-w1:][ind]
    
    fx =  h_0 / lover_2.shape[0]
    lover_2 = cv2.resize(lover_2, (0,0), fx = fx, fy = fx)
    h1, w1, _ = lover_2.shape
    x1 = w // 2 + dist_c
    y1 = y
    h1 = min(h1, h - y1)
    w1 = min(w1, w - x1)
    ind = lover_2[:h1,:w1] != 0
    overlay[y1:y1 + h1, x1:x1 + w1][ind] = lover_2[:h1,:w1][ind]
    return overlay
  #  plot_picture(overlay)

In [566]:
def overlay_image(background, lover_1, lover_2, dist_c = 100):
    h, w, _ = background.shape
    overlay = background.copy()
    
    h_0 = int(h*1)
    y = int(h*0.6)    
    
    fx = h_0 / lover_1.shape[0]
    lover_1 = cv2.resize(lover_1.copy(), (0,0), fx = fx, fy = fx)
    h1, w1, _ = lover_1.shape
    
    fx =  h_0 / lover_2.shape[0]
    lover_2 = cv2.resize(lover_2.copy(), (0,0), fx = fx, fy = fx)
    h2, w2, _ = lover_2.shape    
    
    point_1 = (int(w1*0.5),int(h1*0.5))
    point_2 = (int(w2*0.5),int(h2*0.4))
    
    
    x2 = min(max(w // 2 - dist_c + (w1 - point_1[0]),0), w)
    x1 = min(max(x2 - w1,0), w)
    y1 = min(max(y - point_1[1],0), h)
    y1_0 = max(point_1[1] - y, 0)
    y2 = min(max(y1 + h1 - y1_0,0),h)
    y2_0 = min(max(y1_0 + y2 - y1,0),h1)
    ind = lover_1[y1_0:y2_0,-(x2-x1):] != 0
    overlay[y1:y2,x1:x2][ind] = lover_1[y1_0:y2_0,-(x2-x1):][ind]
    
    x1 = min(max(w // 2 + dist_c - (w2 - point_2[0]),0), w)
    x2 = min(max(x1 + w2,0), w)
    y1 = min(max(y - point_2[1],0), h)
    y1_0 = max(point_2[1] - y, 0)
    y2 = min(max(y1 + h2 - y1_0,0),h)
    y2_0 =  min(max(y1_0 + y2 - y1,0),h2)
    ind = lover_2[y1_0:y2_0,:(x2-x1)] != 0
    overlay[y1:y2,x1:x2][ind] = lover_2[y1_0:y2_0,:(x2-x1)][ind]
    
    
    return overlay


In [567]:
images = []
dist_c = int(w//2 * 0.9)
for i in range(10):
    dist_c = int(dist_c - 50 * (10-i)/10)
    img = overlay_image(background, lover_1, lover_2, dist_c)
    if dist_c < 200:
        ind = heart != 0
        img[ind] = heart[ind]
    images.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))   

In [568]:
imageio.mimsave('1.gif', images, format='GIF')